In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from backtest import *
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pprint
from pprint import PrettyPrinter

In [ ]:
plt.rcParams['figure.figsize'] = [21, 13]
pd.set_option('precision', 10)
pp = PrettyPrinter()

In [ ]:
exchange = 'bybit'
user = 'your_user_name'
settings = load_settings(exchange, user)
s = 'BTCUSD'
n_days = 3

In [ ]:
#load cache if any
agg_trades = pd.read_csv('BTCUSD_agg_trades_bybit_28.0_days_2021-01-12.csv').set_index('trade_id')
adf = agg_trades

In [ ]:
#otherwise
#agg_trades = await load_trades(exchange, user, s, n_days)
#adf = agg_trades

In [ ]:
# look at jackrabbit results, if any

rdf = pd.concat([pd.read_csv('jackrabbit_results_grid/2021-01-12T11:44:50_24_inverse.csv')])
#column_names = ['grid_step', 'markups']
column_names = ['grid_coefficient', 'grid_spacing', 'markups', 'n_close_orders']
#column_names = ['default_qty', 'grid_step', 'margin_limit', 'markups', 'n_close_orders']

rdf.columns = column_names + list(rdf.columns[len(column_names):])
rdfs = rdf.sort_values('gain', ascending=[False])
rdfs = rdfs[~rdfs.gain.duplicated()]
rdfs.loc[:, 'pnl_over_max_margin_cost'] = rdfs.pnl_sum / rdfs.max_margin_cost
#rdfs = rdfs[rdfs.loss_sum == 0.0]
rdfs.head(30)


In [ ]:
# plotting method

def plot_tdf_(df_, tdf_, side_: int = 0, liq_thr=0.1):
    df_.loc[tdf_.index[0]:tdf_.index[-1]].price.plot(style='y-')
    if side_ >= 0:
        longs = tdf_[tdf_.side == 'long']
        le = longs[longs.type == 'entry']
        lc = longs[longs.type == 'close']
        le.price.plot(style='b.')
        longs.pos_price.plot(style='b--')
        if 'close_price' in longs.columns:
            longs.close_price.plot(style='r--')
        lc.price.plot(style='ro')
    if side_ <= 0:
        shrts = tdf_[tdf_.side == 'shrt']
        se = shrts[shrts.type == 'entry']
        sc = shrts[shrts.type == 'close']
        se.price.plot(style='r.')
        shrts.pos_price.plot(style='r--')
        if 'close_price' in shrts.columns:
            shrts.close_price.plot(style='b--')
        sc.price.plot(style='bo')
    if 'liq_price' in tdf_.columns:
        tdf_.liq_price.where((tdf_.price - tdf_.liq_price).abs() / tdf_.price < liq_thr, np.nan).plot(style='k--')


In [ ]:
# crop trades if desired

age_limit = (time() - 60 * 60 * 24 * 4) * 1000
adf = adf[adf.timestamp > age_limit]
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
n_days


In [ ]:
{'n_closes': 14520, 'pnl_sum': 0.0019708674546522966, 'loss_sum': 0.0, 'max_margin_cost': 0.00014454813469976551, 'average_daily_gain': 1.029694460329125, 'gain': 2.015911059099122, 'n_trades': 30557, 'closest_liq': 0.06959398293567425, 'biggest_pos_size': 441.0} 

In [ ]:
# prep df for backtesting
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
print('ndays', n_days)
df = prep_df(adf)
print(len(df), len(df) / len(adf))
df.price.iloc[::100].plot()

In [ ]:
# bybit
settings = {
    'inverse': True,
    'maker_fee': -0.00025,
    'price_step': 0.5,
    'qty_step': 1.0,
    'symbol': 'BTCUSD',
    'n_entry_orders': 10,
    'leverage': 100,
    'min_qty': 1.0,

    'break_on_loss': False,
    'compounding': False,
    'min_markup': 0.0002,
    'margin_limit': 0.0019,
    'default_qty': 2.0,
    
    'max_markup': 0.0159,
    'n_close_orders': 17,
    'liq_diff_threshold': 0.01,
    'stop_loss_pos_reduction': 0.02,
    
    'dynamic_grid': True, # true for dynamic grid style, false for static grid style
    
    # dynamic specific settings
    'grid_coefficient': 0.0,
    'grid_spacing': 0.0034,
    
    # static specific settings
    'grid_step': 116.5,
}

In [ ]:
print('approx default grid step', df.price.mean() * settings['grid_spacing'])
sort_dict_keys(settings)

In [ ]:
start_time = time()
trades = backtest(df, settings)
print(f'seconds elapsed {time() - start_time:.2f}')

In [ ]:
# analyze results
tdf = pd.DataFrame(trades).set_index('trade_id')
longs = tdf[tdf.side == 'long']
shrts = tdf[tdf.side == 'shrt']
le = longs[longs.type == 'entry']
lc = longs[longs.type == 'close']
se = shrts[shrts.type == 'entry']
sc = shrts[shrts.type == 'close']

margin_max = (tdf.pos_size.abs() / tdf.pos_price).max() / settings['leverage']
biggest_pos_size = tdf.pos_size.abs().max()
pnl_sum = tdf.pnl.sum()
loss_sum = tdf[tdf.pnl < 0.0].pnl.sum()
gain = (settings['margin_limit'] + pnl_sum) / settings['margin_limit']
closest_liq = ((tdf.price - tdf.liq_price).abs() / tdf.price).min()
n_days = (adf.timestamp.iloc[-1] - adf.timestamp.iloc[0]) / 1000 / 60 / 60 / 24
average_daily_gain = gain ** (1 / n_days)
closes = tdf[tdf.type == 'close']
print('margin_max', margin_max)
print('biggest qty', tdf.qty.abs().max())
print('pnl_sum', pnl_sum)
print('loss sum', loss_sum)
print('gain', gain)
print('n_days', n_days)
print('average_daily_gain', average_daily_gain)
print('n trades', len(tdf))
print('n closes', len(closes))
print('biggest_pos_size', biggest_pos_size)
print('closest liq', closest_liq)

In [ ]:
plot_tdf_(df, tdf, liq_thr=0.08)

In [ ]:
# visualize behavior
step = 120
i = -step

In [ ]:
i += step
tdfc = tdf.iloc[i:i+step]
plot_tdf_(df, tdf.iloc[i:i+step], liq_thr=0.06)

In [ ]:
tdfcj = tdfc.join(pd.Series(tdfc.price.diff(), name='price_diff'))
tdfcj.head(60)

In [ ]:
tdfcj.tail(60)

In [ ]:
tdf.pnl.cumsum().plot()

In [ ]:
tdf.pos_size.plot()

In [ ]:
closest_liqs = ((tdf.liq_price - tdf.price).abs() / tdf.price).sort_values()
closest_liqs.head()

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(closest_liqs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
# inspect biggest trades
qty_abs = tdf.qty.abs().sort_values(ascending=False)
qty_abs.head(10)

In [ ]:
i = 0
iloc_ = tdf.index.get_loc(qty_abs.index[i])
iminus = 400
iplus = 20
tdfc = tdf.iloc[max(0, iloc_-iminus):min(iloc_+iplus, len(tdf) - 1)]
plot_tdf_(df, tdfc)

In [ ]:
ik = -1
stepp = 60

In [ ]:
ik += 1
tdfc.iloc[stepp * ik:stepp * (ik + 1)]